In [5]:
import numpy as np
from sys import path
path.append('../tools')
from tools import logsig

In [6]:
def get_expected_output(X):
    return logsig(np.dot(logsig(X * np.array([10, 10]) + np.array([-5, 5])), np.array([1, 1])) + -1)

In [7]:
vec_get_y = np.vectorize(get_expected_output)

In [12]:
x_train = np.linspace(-2, 2, 82)
y_train = vec_get_y(x_train)

In [13]:
x_train = x_train.reshape(len(x_train), 1)
y_train = y_train.reshape(len(y_train), 1)

In [19]:
# Create the model with keras and tensorflow
# Adapted from : https://keras.io/guides/custom_train_step_in_tensorflow/

#Imports. Need to do this for some reason
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import tensorflow as tf
import keras
from keras import layers

In [82]:
# Create the model
class CustomModel(keras.Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.search_direction = None
        self.beta = None
        self.old_flattened_gradients = None
        
    def train_step(self, data):
        self.optimizer.learning_rate.assign(.1)
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data
        
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compute_loss(y=y, y_pred=y_pred)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        flattened_gradients = tf.concat([tf.reshape(gradient, [-1]) for gradient in gradients], 0)

        if self.search_direction is None:
            self.search_direction = -gradients
            self.flattened_search_direction = -flattened_gradients
            self.old_flattened_gradients = -flattened_gradients
        else:
            self.beta = 

        # Find interval for the best learning rate

        # Update weights
        self.optimizer.apply(-self.search_direction, trainable_vars)

        # Update metrics (includes the metric that tracks the loss)
        for metric in self.metrics:
            if metric.name == "loss":
                metric.update_state(loss)
            else:
                metric.update_state(y, y_pred)

        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

In [83]:
# Construct and compile an instance of CustomModel
inputs = keras.Input(shape=(1,))
x = keras.layers.Dense(2)(inputs)
outputs = keras.layers.Dense(1)(x)
model = CustomModel(inputs, outputs)
model.compile(optimizer="SGD", loss="mse")

model.fit(x_train, y_train, epochs=3)

Epoch 1/3
None
Tensor("concat:0", shape=(7,), dtype=float32)
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step - loss: 0.3984
Epoch 2/3
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 773us/step - loss: 0.0059
Epoch 3/3
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 906us/step - loss: 0.0031
